# Playing With Celes Data

## Preparing environment

### Set the PySpark environment variables

In [5]:
import os
os.environ['SPARK_HOME'] = "/Users/efuentesamin/spark"
os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = 'lab'
os.environ['PYSPARK_PYTHON'] = 'python'
os.environ['SPARK_LOCAL_IP'] = '192.168.1.13'

### Install PySpark

In [4]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.4 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 5.7 MB/s eta 0:00:00ta 0:00:01
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=1e2bb37d6cc65c4100f949d5d98f3fc0523d806c54f9d7b45c1d0763246b890a
  Stored in directory: /Users/efuentesamin/Library/Caches/pip/wheels/95/13/41/f7f135ee114175605fb4f0a89e7389f3742aa6c1e1a5bcb657
Successfully built pyspark
Note: you may need to restart the kernel to use updated packages.


### Import PySpark Session

In [6]:
from pyspark.sql import SparkSession

### Create a SparkSession

In [7]:
spark = SparkSession.builder \
    .appName("Playing-With-Celes-Data") \
    .getOrCreate()
sc = spark.sparkContext
sc.setLogLevel("WARN")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/08 18:34:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Playing with data

### Load datamart

In [8]:
df = spark.read.parquet('./data')

24/06/08 18:34:46 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [9]:
df.show(1, vertical=True)

-RECORD 0----------------------------
 KeySale      | 5540A1C6-1E83-EE1... 
 KeyDate      | 2023-11-14           
 KeyStore     | 1|007                
 KeyWarehouse | 1|1008               
 KeyCustomer  | 1|POS|32694425       
 KeyProduct   | 1|60925              
 KeyEmployee  | 1|417                
 KeyCurrency  | 1|COP                
 KeyDivision  | 1                    
 KeyTicket    | 1|F01-00322372       
 KeyCedi      | 1|1                  
 TicketId     | F01-00322372         
 Qty          | 1.0                  
 Amount       | 28487.4              
 CostAmount   | 8557.97              
 DiscAmount   | 0.0                  
 Tickets      | {5540A1C6-1E83-EE... 
 Products     | {CORPORAL        ... 
 Customers    | {1|POS|32694425, ... 
 Employees    | {1|417, 417, 1, G... 
 Stores       | {1|007, 1, 007, P... 
 Divisions    | {1, 1, DROGUERIAS... 
 Time         | {2023-11-14, 2023... 
 Cedis        | {1|1, 1, 1, Cedi ... 
only showing top 1 row



### Get schema

In [10]:
df.printSchema()

root
 |-- KeySale: string (nullable = true)
 |-- KeyDate: date (nullable = true)
 |-- KeyStore: string (nullable = true)
 |-- KeyWarehouse: string (nullable = true)
 |-- KeyCustomer: string (nullable = true)
 |-- KeyProduct: string (nullable = true)
 |-- KeyEmployee: string (nullable = true)
 |-- KeyCurrency: string (nullable = true)
 |-- KeyDivision: string (nullable = true)
 |-- KeyTicket: string (nullable = true)
 |-- KeyCedi: string (nullable = true)
 |-- TicketId: string (nullable = true)
 |-- Qty: double (nullable = true)
 |-- Amount: double (nullable = true)
 |-- CostAmount: double (nullable = true)
 |-- DiscAmount: double (nullable = true)
 |-- Tickets: struct (nullable = true)
 |    |-- KeySale: string (nullable = true)
 |    |-- KeyDivision: string (nullable = true)
 |    |-- KeyDate: date (nullable = true)
 |    |-- OrderDate: date (nullable = true)
 |    |-- KeyEmployee: string (nullable = true)
 |    |-- KeyCustomer: string (nullable = true)
 |    |-- KeyStore: string (nul

### Operations

In [19]:
df.orderBy(df.KeyDate.asc()).select(df.KeyDate).show(1)

+----------+
|   KeyDate|
+----------+
|2023-10-03|
+----------+
only showing top 1 row



In [20]:
df.orderBy(df.KeyDate.desc()).select(df.KeyDate).show(1)

+----------+
|   KeyDate|
+----------+
|2023-11-30|
+----------+
only showing top 1 row



In [12]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [15]:
dates = ('2023-10-03', '2023-11-30')

In [22]:
df.select('KeySale', 'KeyDate', 'Amount') \
    .filter((col('KeyEmployee') == '1|591') & (col('KeyDate').between(*dates))) \
    .orderBy(desc('KeyDate')) \
    .offset(0).limit(10) \
    .show()

+--------------------+----------+--------+
|             KeySale|   KeyDate|  Amount|
+--------------------+----------+--------+
|9ED3D5E9-A88F-EE1...|2023-11-30|12941.18|
|4CF0C2FB-848F-EE1...|2023-11-30|54537.81|
|48B126A6-8D8F-EE1...|2023-11-30| 3781.51|
|1F0ECC78-8E8F-EE1...|2023-11-30| 2352.94|
|0D3A8679-878F-EE1...|2023-11-30| 5798.32|
|C0E0699D-A98F-EE1...|2023-11-30| 11500.0|
|6B5BD463-AC8F-EE1...|2023-11-30| 8319.33|
|1F0ECC78-8E8F-EE1...|2023-11-30| 3781.51|
|0D3A8679-878F-EE1...|2023-11-30| 3781.51|
|4130B566-C08F-EE1...|2023-11-30| 8235.29|
+--------------------+----------+--------+



In [23]:
df.groupBy('KeyStore', 'Stores.StoreName') \
    .agg(sum('Amount').alias('TotalAmount'), avg('Amount').alias('AvgAmount')) \
    .orderBy(asc('Stores.StoreName')) \
    .offset(0).limit(10) \
    .show()

+--------+----------+--------------------+------------------+
|KeyStore| StoreName|         TotalAmount|         AvgAmount|
+--------+----------+--------------------+------------------+
|   1|005| AMERICANO| 1.805872628700007E8|11743.221671868949|
|   1|015|BOCAGRANDE| 1.765580215500002E8|15157.797179773368|
|   1|098|CALLCENTER|1.5204748104000017E8| 27524.88795076035|
|   1|008|  CALLE 29|2.0222230642999938E8| 14063.72532373596|
|   1|006|   CALLE30|2.1416953741000015E8|14698.341734266703|
|   1|022|   CALLE79|1.7855922616000035E8|14105.318442215053|
|   1|004|   CALLE84|4.4964245264999795E8|16536.444141443786|
|   1|021|   CALLE93| 2.803403881500005E8|14454.260796597087|
|   1|003|    CENTRO| 2.362186489999996E8|13746.429760242063|
|   1|009| EL AMPARO| 2.374517804700001E8| 11154.77899516137|
+--------+----------+--------------------+------------------+



In [24]:
df.groupBy('KeyProduct', 'Products.ProductName') \
    .agg(sum('Amount').alias('TotalAmount'), avg('Amount').alias('AvgAmount')) \
    .orderBy(asc('Products.ProductName')) \
    .offset(0).limit(10) \
    .show()

+----------+--------------------+------------------+------------------+
|KeyProduct|         ProductName|       TotalAmount|         AvgAmount|
+----------+--------------------+------------------+------------------+
|   1|62423|ACACIA DE LA INDI...|1337962.3400000008| 6137.441926605508|
|   1|62425|ACACIA DE LA INDI...|         641138.22| 3771.401294117647|
|   1|62426|ACACIA DE LA INDI...|         513403.54|        12835.0885|
|   1|55136|ACAROL LOCION CUT...|          777400.0|           29900.0|
|   1|62348|ACEITE AMBIENTAL ...|2051322.8999999997|12508.066463414632|
|   1|62344|ACEITE AMBIENTAL ...|1541658.8699999996| 9882.428653846151|
|   1|60322|ACEITE CORP NARAN...| 4646322.529999996| 7629.429441707711|
|   1|60323|ACEITE CORP NARAN...| 5900296.720000003|17934.032583586635|
|   1|60253|ACEITE CORP NARAN...| 779159.6799999999| 86573.29777777777|
|   1|60324|ACEITE CORPORAL D...| 5873549.999999994|7520.5505761843715|
+----------+--------------------+------------------+------------

In [25]:
df.groupBy('KeyEmployee', 'Employees.EmployeeName') \
    .agg(sum('Amount').alias('TotalAmount'), avg('Amount').alias('AvgAmount')) \
    .orderBy(asc('Employees.EmployeeName')) \
    .offset(0).limit(10) \
    .show()

+-----------+--------------------+--------------------+------------------+
|KeyEmployee|        EmployeeName|         TotalAmount|         AvgAmount|
+-----------+--------------------+--------------------+------------------+
|      1|591|ACOSTA FLOREZ FER...| 3.560228748999997E7|15378.957879049663|
|      1|354|ACOSTA PATERNINA ...|1.4547493619999988E7|11130.446534047427|
|    1|12208|ALTAMAR FANDIÑO P...|             3697.48|           3697.48|
|      1|362|ALVAREZ SANCHEZ E...|          6837192.24| 17992.61115789474|
|    1|17878|ALVAREZ VEGA BETT...|  3.40040724700001E7| 21952.27402840549|
|    1|15934| ARANGO LOPEZ NATALI|                 0.0|               0.0|
|    1|16473|ARIZA MARRIAGA LU...|  2.70248578000001E7|17674.857946370244|
|     1|1141|ARRIETA ALVAREZ J...| 9.704634070999984E7| 14497.51131012845|
|    1|17514|ARRIETA MUÑOZ MIT...|3.6227331560000114E7| 21222.80700644412|
|    1|13496|BADILLO MARQUEZ M...|                 0.0|               0.0|
+-----------+------------